设计和实现一个程序要考虑的最重要的事是**程序应该产生可靠的结果**。比如，我们希望我们的银行卡余额被正确地计算。我们希望我们的汽车里的喷油器能喷出合适的燃料。我们既不想飞机坠毁，也不想操作系统崩溃。

有时，**性能**是正确性的一个重要方面。

对于需要实时运行的程序来说，这是最显然的。一个警告飞机潜在障碍物的程序需要在碰到障碍物之前发出警告。

性能也会影响许多非实时应用的效用。当评估数据库的性能时，每分钟完成的事务数是一个重要的度量。用户关心手机上开启一个应用所需的时间。生物学家关心它们的系统发育推断花费的时间。

写高效率的程序不容易。最直接的解往往不是最有效的。计算效率高的程序通常会利用微妙的算法，使其很难被理解。为了减小**计算复杂度**，程序员通常会增加**概念复杂度**。为了以合理的方式做到这一点，我们需要理解如何评估一个程序的计算复杂度。这就是本章的主题。

## 9.1 思考计算复杂度

问题1：下面的程序将花费多长时间来运行？

In [1]:
def f(i):
    """假设i是一个整数，且i>=0"""
    answer = 1
    while i >= 1:
        answer *= i
        i = i - 1
    return answer

print(f(5))

120


我们可以对某些输入运行程序，并计时。但是那并不是特别有用，因为结果取决于以下三个因素：
- 运行它的计算机的速度
- 在那台机器上的Python实现的效率
- 输入值

我们使用更抽象的时间测量来解决前两个问题。不用毫秒数来测量时间，我们使用**一个程序执行的基本步骤的数量来衡量时间**。

为了简单起见，我们使用**随机访问机器**作为我们的计算模型。
- 在一台随机访问机器上，步骤是被顺序执行的，一次执行一个步骤。
- 一个步骤就是一个花费固定时间的操作，比如绑定一个变量到一个对象，作比较，执行一个算术操作，访问内存中的一个对象等。

既然我们有了更抽象的测量时间的方式，那么我们转到输入值依赖的问题上来。我们处理这个问题从将时间复杂度表示为单个数转移到**建立时间复杂度跟输入大小的关联**上。这就允许我们通过讨论**每个算法随着输入大小增长如何变化**来比较两个算法的效率。

当然，一个算法实际运行时间不仅取决于输入的大小，也取决于输入值。比如，

问题2：考虑线性搜索算法可被实现为：

In [2]:
def linearSearch(L,x):
    for e in L:
        if e == x:
            return True
    return False

假设L是一个包含100万个元素的列表，考虑调用`linearSearch(L,3)`。

如果L中的第一个元素是3，则`linearSearch`立即返回True。
如果3不在L中，则`linearSearch`在返回False前要检测所有100万个元素。

一般来说，有3大类情形要考虑：

- 最好情形运行时间是输入最有利时的运行时间，即在给定大小的所有可能输入中算法的最小运行时间。比如，linearSearch的最好运行时间独立于列表L的大小。

- 最坏情形运行时间是在给定大小的所有输入中算法的最大运行时间。比如，linearSearch的最坏情形运行时间线性于列表L的大小。

- 平均情形运行时间是在给定大小的所有输入中算法的平均运行时间。如果你有关于输入值分布的先验知识(比如x在L中的概率是90%)，那么可以把这一点考虑进去。

人们通常关注的是最坏情形。所有工程师都有一个共同的信念，即莫非定律：如果一件事情可能出错，则它将会出错。最坏情形提供了运行时间的上界。在计算时间有限制的情况下，这是非常关键的。仅仅知道“大多数时候”空中交通管制系统在即将发生的碰撞之前就发出警告是不够的。

示例：阶乘函数的迭代实现的最坏情形运行时间

In [3]:
def fact(n):
    """Assumes n is a natural number
       Returns n!"""
    answer = 1
    while n > 1:
        answer *= n
        n -= 1
    return answer

分析：运行这个程序大约需要2+5n步，其中2步包括1步的初始的赋值语句和1步的返回语句，5步包括1步的循环测试、2步的while循环内的第一条赋值语句、2步的循环内的第2条赋值语句。所以，如果n是1000，那么该函数将执行大约5002步。

当n变大时，担心5n和5n+2的差有点蠢。因为这个原因，所以当推断运行时间时，通常会忽略额外的常数。我们应该关心计算是需要1000步还是5000步吗？乘法系数可能很重要。搜索引擎是需要半秒还是2.5秒来为查询提供服务可能是人们使用该搜索引擎还是去竞争对手之间的区别。

另一方面，当比较两种不同的算法时，通常的情形是即使是乘法系数也不相关。回想一下，在第3章中，我们研究了两种算法用于找到浮点数平方根的近似值，即穷举枚举和二分搜索。基于这些算法的函数如图9.1和图9.2所示。

In [2]:
def squareRootExhaustive(x, epsilon):
    """Assumes x and epsilon are positive floats & epsilon < 1
       Returns a y such that y*y is within epsilon of x"""
    step = epsilon**2
    ans = 0.0
    numCount = 0
    while abs(ans**2 - x) >= epsilon and ans*ans <= x:
        ans += step
        numCount+=1
    
    if ans*ans > x:
        raise ValueError
    print('bi loop count=', numCount)
    return ans

# squareRootExhaustive(100, 0.0001)

In [3]:
def squareRootBi(x, epsilon):
    """Assumes x and epsilon are positive floats & epsilon < 1
       Returns a y such that y*y is within epsilon of x"""
    low = 0.0
    high = max(1.0, x)
    ans = (high + low)/2.0
    numCount = 0
    while abs(ans**2 - x) >= epsilon:
        if ans**2 < x:
            low = ans
        else:
            high = ans
        ans = (high + low)/2.0
        numCount+=1
    
    print('bi loop count=', numCount)
    return ans

squareRootBi(100, 0.0001)

bi loop count= 22


10.000002384185791

我们看到穷举法是如此缓慢，以至于对于`x`和`epsilon`的许多组合是不切实际的。例如，评估`squareRootExhaustive(100, 0.0001)`需要大约10亿次`while`循环迭代。相比之下，计算`squareRootBi(100, 0.0001)`需要大约20次迭代一个稍微复杂的while循环。 当迭代次数的差异如此之大时，循环中有多少指令并不重要。 即，乘法常数是无关紧要的。

## 9.2 渐近符号
我们**渐近符号**来提供一种正式的方式来讨论算法的运行时间与其输入大小之间的关系。潜在的动机是几乎任何算法在小输入上运行时都足够有效。我们通常需要担心的是算法在非常大的输入上运行时的效率。作为“非常大”的代表，渐近符号描述了随着输入的大小接近无穷大时算法的复杂性。

考虑图9.3中的代码：

In [7]:
def f(x):
    """Assume x is an int > 0"""
    ans = 0
    #Loop that takes constant time
    for i in range(1000):
        ans += 1
    print('Number of additions so far', ans)
    #Loop that takes time x
    for i in range(x):
        ans += 1
    print('Number of additions so far', ans)
    #Nested loops take time x**2
    for i in range(x):
        for j in range(x):
            ans += 1
            ans += 1
    print('Number of additions so far', ans)
    return ans

分析：如果假设执行每行代码花费一个单位的时间，那么这个函数的运行时间可被描述为$1000+x+2x^2$，其中常数1000对应的是第一个循环的执行次数，x对应的是第二个循环被执行的次数，$2x^2$对应的是执行嵌套循环中两条语句花费的时间。因此，调用`f(10)`将会输出：

In [8]:
f(10)

Number of additions so far 1000
Number of additions so far 1010
Number of additions so far 1210


1210

调用`f(1000)`会输出：

In [9]:
f(1000)

Number of additions so far 1000
Number of additions so far 2000
Number of additions so far 2002000


2002000

对小点的x值，常数项占主导。如果x是10，超过80%的步数花费在第一个循环上。但是，如果x是1000，前两个循环中的每个仅花费整个步数的0.05%。当x是1000000时，第一个循环占据整个时间的0.00000005%，第二个循环占据整个时间的0.00005%。2,000,001,001,000步中有2,000,000,000,000步花费在内部的for循环上。

显然，通过仅考虑内部循环（即二次项），我们可以得到一个关于**这段代码在非常大的输入上运行需要多长时间**这一有意义的概念。我们应该关心循环花费$2x^2$而不是是$x^2$吗？如果您的计算机每秒执行大约1亿步，评估`f`大约需要5.5小时。如果我们可以将复杂度降低到$x^2$步，大约需要2.25小时。在任何一种情况下，困境都是一样的：我们可能应该寻找更有效的算法。

这种分析导致我们使用以下经验法则来描述算法的渐近复杂度:
- 如果运行时间为多项之和时，保留增长速度最大的项，删除其他项。
- 如果剩下的项是乘积，去掉任何常数。

最常用的渐近符号是大O符号。使用大O符号来给出一个函数渐近增长(增长阶)的上界。
- $f(x)\in O(x^2)$表示从渐近的意义上讲，函数`f`的增长不快于二次多项式$x^2$。
- $f(x)$是$O(x^2)$表示从渐近最坏情形的运行时间来看，$x^2$既是上界又是下界。